# Steps to generate recommendations
1. Run StartDaemons.ipynb to setup the servers
2. Put your username and source in the cell below. For example, setting username="Fro116" and source="mal" will generate recommendations for the user "Fro116" on the site "myanimelist.net"
3. Run the script to generate an html page of recommendations

In [ ]:
username = ""
source = ""

In [ ]:
daemon = True
sources = [
    "mal",
    "anilist",
    "kitsu",
    "animeplanet",
    "training",
]

In [ ]:
assert username, "Must provide a username"
assert source in sources, "Must provide a list source"

In [ ]:
import logging
import os
import shutil
import subprocess

import papermill as pm

In [ ]:
ALL_MEDIUMS = ["manga", "anime"]

In [ ]:
logging.disable(logging.ERROR)

In [ ]:
data_path = os.path.abspath("../../data/recommendations")
if not os.path.exists(data_path):
    os.mkdir(data_path)
if source == "training":
    rec_dir = os.path.join(data_path, username.split("@")[0], username.split("@")[1])
else:
    rec_dir = os.path.join(data_path, source, str(username))

In [ ]:
def run_julia_script(outdir, name, port, *args):
    pwd = os.getcwd()
    try:
        os.chdir(outdir)
        if daemon:
            os.system(
                f"julia --startup-file=no -e 'using DaemonMode; runargs({port})' {name} {' '.join(args)}"
            )
        else:
            os.system(f"julia {name} {' '.join(args)}")
    finally:
        os.chdir(pwd)

In [ ]:
def run_python_script(outdir, script, **kwargs):
    pwd = os.getcwd()
    try:
        os.chdir(outdir)
        procs = []
        for medium in ALL_MEDIUMS:
            cmdlist = [
                "papermill",
                script,
                "/dev/null",
                "-p",
                "medium",
                medium,
            ]
            for k, v in kwargs.items():
                cmdlist.append("-p")
                cmdlist.append(k)
                cmdlist.append(v)
            procs.append(
                subprocess.Popen(
                    cmdlist, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT
                )
            )
        for p in procs:
            p.wait()
    finally:
        os.chdir(pwd)

## RecIngress

In [ ]:
if os.path.exists(rec_dir):
    shutil.rmtree(rec_dir)

In [ ]:
%%time
procs = []
for medium in ALL_MEDIUMS:
    port = 3020 + sources.index(source)
    cmdlist = [
        "curl",
        f"http://localhost:{port}/query?username={username}&medium={medium}",
    ]
    procs.append(
        subprocess.Popen(cmdlist, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
    )
    for p in procs:
        p.wait()

In [ ]:
original_source = source
username = str(username)
if source == "training":
    source, username = username.split("@")

In [ ]:
%%time
run_python_script(
    "../RecIngress", "ImportMediaLists.ipynb", username=username, source=source
)

In [ ]:
%%time
run_python_script(
    "../RecIngress", "ProcessMediaLists.ipynb", username=username, source=source
)

In [ ]:
%%time
run_python_script(
    "../RecIngress",
    "GenerateMediaSplits.ipynb",
    username=username,
    source=source,
    original_source=original_source,
)

## RecEgress

In [ ]:
%%time
run_julia_script(
    "../InferenceAlphas", "RunAllScripts.jl", 3000, username, source, str(daemon)
)

In [ ]:
%%time
run_julia_script("Recommendations", "Recommendations.jl", 3007, username, source)

In [ ]:
def runcmd(cmdlist):
    subprocess.Popen(
        cmdlist, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT
    ).wait()


def export_html(username, source):
    pwd = os.getcwd()
    try:
        os.chdir("./Recommendations")
        outfn = os.path.join(rec_dir, "Recommendations.ipynb")
        runcmd(
            [
                "papermill",
                "Recommendations.ipynb",
                outfn,
                "--prepare-only",
                "-p",
                "username",
                username,
                "-p",
                "source",
                source,
            ]
        )
        runcmd(
            ["jupyter", "nbconvert", "--to", "html", "--execute", "--no-input", outfn]
        )
    finally:
        os.chdir(pwd)

In [ ]:
%%time
export_html(username, source)